# Chapter9.SparkSQL

### 9.1连接SparkSQL
- 1.可选 使用带有HIVE支持的SparkSQL  需要拷贝 hive-site.xml到$Spark_HOME/conf 如果没部署好Hive则SsparkSQL会自己创建Hive元数据仓库

- 2.可选 文件系统 默认使用HDFS,需要拷贝hdfs-site.xml到$Spark_HOME/conf 没有则使用本地文件系统

### 9.2在Applicaition中使用SparkSQL


#### 初始化SparkSQL

In [1]:
from pyspark.sql import HiveContext, Row
from pyspark.sql import SQLContext, Row
from pyspark import SparkConf, SparkContext
sc = SparkContext('local[*]', 'HiveTest')
hiveCtx = HiveContext(sc)

#### 创建表测试

In [2]:
hiveCtx.sql("""
create table page_view  
(  
page_id bigint comment '页面ID',  
page_name string comment '页面名称',  
page_url string comment '页面URL'  
)  
""")

DataFrame[]

In [3]:
hiveCtx.sql("insert into page_view values(1,'Niko','www.baidu.com')")

DataFrame[]

In [7]:
hiveCtx.sql("select * from page_view").collect()

[Row(page_id=1, page_name='Niko', page_url='www.baidu.com')]

#### 基本查询

In [10]:
import os
parent_path = os.path.abspath('../')
parent_path

'/Users/cool/OneDrive/002_projects/learning-spark'

In [15]:
inputFile = 'file://{}/files/testweet.json'.format(parent_path)
input = hiveCtx.read.json(inputFile)
# 注册输入的SchemaRDD(现在更名为DataFrame)
input.registerTempTable('tweets')

In [19]:
# 依据retweetCount选出推文
topTweets = hiveCtx.sql("select text,retweetCount from tweets order by retweetCount limit 10")

In [20]:
input.printSchema()

root
 |-- contributorsIDs: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- createdAt: string (nullable = true)
 |-- currentUserRetweetId: long (nullable = true)
 |-- hashtagEntities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: long (nullable = true)
 |-- inReplyToStatusId: long (nullable = true)
 |-- inReplyToUserId: long (nullable = true)
 |-- isFavorited: boolean (nullable = true)
 |-- isPossiblySensitive: boolean (nullable = true)
 |-- isTruncated: boolean (nullable = true)
 |-- mediaEntities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- retweetCount: long (nullable = true)
 |-- source: string (nullable = true)
 |-- text: string (nullable = true)
 |-- urlEntities: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- user: struct (nullable = true)
 |    |-- createdAt: string (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- descriptionURL

In [21]:
topTweets.collect()

[Row(text='Adventures With Coffee, Code, and Writing.', retweetCount=0)]

#### SchemaRDD

In [42]:
topTweets.show()

+--------------------+------------+
|                text|retweetCount|
+--------------------+------------+
|Adventures With C...|           0|
+--------------------+------------+



In [43]:
topTweeText = topTweets.filter(lambda row: row.text)
topTweeText.collect()

TypeError: condition should be string or Column

#### 缓存

In [46]:
hiveCtx.cacheTable('tweets')
topTweets.persist()

DataFrame[text: string, retweetCount: bigint]

### 9.3 读取和存储数据

In [56]:
rows = hiveCtx.sql("select * from tweets")

In [59]:
rows.columns

['contributorsIDs',
 'createdAt',
 'currentUserRetweetId',
 'hashtagEntities',
 'id',
 'inReplyToStatusId',
 'inReplyToUserId',
 'isFavorited',
 'isPossiblySensitive',
 'isTruncated',
 'mediaEntities',
 'retweetCount',
 'source',
 'text',
 'urlEntities',
 'user',
 'userMentionEntities']

In [32]:
rows = hiveCtx.sql("select key , value from mytable")
keys = rows.map(lambda row : row[0])

AnalysisException: 'Table not found: mytable; line 1 pos 24'

#### Json

In [75]:
rows = hiveCtx.sql("select user. followersCount from tweets")
rows.collect()

[Row(followersCount=1231)]

#### 基于RDD

In [84]:
user = sc.parallelize([Row(name='NikoBelic', age = '18')])
userDateFrame = hiveCtx.createDataFrame(user)
userDateFrame.registerTempTable('user')

In [85]:
res = hiveCtx.sql('select * from user')
res.collect()

[Row(age='18', name='NikoBelic')]

### 9.4 JDBC/ODBC 服务器
- start-thriftserver.sh
- sudo beeline -u jdbc:hive2://localhost:10000


### 9.5 用户自定义函数UDF

In [86]:
# 求推文长度
from pyspark.sql.types import IntegerType
hiveCtx.registerFunction('strLenPython', lambda x: len(x), IntegerType())
lengthSchemaRDD = hiveCtx.sql('select strLenPython(text) from tweets limit 10')
lengthSchemaRDD.collect()

[Row(strLenPython(text)=42)]

### 9.6 SprakSQL性能